# Tools setup

In [ ]:
import os
import shutil

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

!rm -r sample_data


In [ ]:
# to hide cell output
%%capture

#!pip install tensorflow==2.*
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2
!git clone https://github.com/tensorflow/models.git

# Compile protos.
%cd models/research
!protoc object_detection/protos/*.proto --python_out=.

%cd /content
!pip install cython

!git clone https://github.com/cocodataset/cocoapi.git
%cd cocoapi/PythonAPI
!make

%cp -r pycocotools /content/models/research

%cd /content/models/research
%cp object_detection/packages/tf2/setup.py .
!python -m pip install .

# Test
#!python object_detection/builders/model_builder_tf2_test.py


In [ ]:
workspace_path = '/content/drive/MyDrive/CVproject_Stenosis/workspace'
models_path = '/content/drive/MyDrive/CVproject_Stenosis/workspace/models'
pre_trained_models_path = '/content/drive/MyDrive/CVproject_Stenosis/workspace/pre_trained_models'
exported_models_path = '/content/drive/MyDrive/CVproject_Stenosis/workspace/exported_models'
tfrecord_output_path = '/content/drive/MyDrive/CVproject_Stenosis/workspace/data'
train_record_path = '/content/drive/MyDrive/CVproject_Stenosis/workspace/data/train.record'
validation_record_path = '/content/drive/MyDrive/CVproject_Stenosis/workspace/data/validation.record'
test_record_path = '/content/drive/MyDrive/CVproject_Stenosis/workspace/data/test.record'
label_map_path = '/content/drive/MyDrive/CVproject_Stenosis/workspace/data/label_map.pbtxt'
dataset_path = '/content/drive/MyDrive/CVproject_Stenosis/Dataset'
images_path = '/content/drive/MyDrive/CVproject_Stenosis/Dataset/Images'
csv_path = '/content/drive/MyDrive/CVproject_Stenosis/Dataset/CSV'

object_detection_path = '/content/models/research/object_detection'
model_main_tf2 = 'model_main_tf2.py'
exporter_main_v2 = 'exporter_main_v2.py'

#copy_path1 = os.path.join(object_detection_path, model_main_tf2)
#copy_path2 = os.path.join(workspace_path, model_main_tf2)
#shutil.copy(copy_path1, copy_path2)

#copy_path1 = os.path.join(object_detection_path, exporter_main_v2)
#copy_path2 = os.path.join(workspace_path, exporter_main_v2)
#shutil.copy(copy_path1, copy_path2)

num_train_samples = 6700
num_eval_samples = 800
num_test_samples = 845

# SSD ResNet50 V1 FPN 640x640 - Testing

## V1

In [ ]:
model_tech_name = 'ssd_resnet50_v1_fpn_640x640_coco17_tpu-8'
model_name = 'ssd_resnet50'
version_name = 'v1'

model_path = os.path.join(models_path, model_name)
version_path = os.path.join(model_path, version_name)

train_log_path = os.path.join(version_path, 'train')
eval_log_path = os.path.join(version_path, 'eval')
log_path = version_path

pre_trained_conf_path = os.path.join(pre_trained_models_path, model_tech_name, 'pipeline.config')
model_conf_path = os.path.join(models_path, model_name, version_name, 'pipeline.config')
checkpoint_path = os.path.join(pre_trained_models_path, model_tech_name, 'checkpoint/ckpt-0')

model_name_version = model_name + '_' + version_name
exported_model_path = os.path.join(exported_models_path, model_name_version)
exported_model_conf_path = os.path.join(exported_models_path, model_name_version, 'pipeline.config')

batch_size = 4
num_epoch = 15
num_steps = int(num_train_samples / batch_size) * num_epoch
num_steps += 100
print(num_steps)
# ------------------------------------- END MODEL SETTINGS --------------------------------------- #

In [ ]:
!export CUDA_VISIBLE_DEVICES=0
!nvidia-smi -L

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir $log_path/eval

In [ ]:
# VALIDATION - Test set
test_conf_path = version_path + "/test.config"
!python /content/drive/MyDrive/CVproject_Stenosis/workspace/model_main_tf2.py \
  --pipeline_config_path=$test_conf_path \
  --model_dir=$version_path \
  --checkpoint_dir=$version_path \
  --sample_1_of_n_eval_examples=1 \

## V2

In [ ]:
model_tech_name = 'ssd_resnet50_v1_fpn_640x640_coco17_tpu-8'
model_name = 'ssd_resnet50'
version_name = 'v22'

model_path = os.path.join(models_path, model_name)
version_path = os.path.join(model_path, version_name)

train_log_path = os.path.join(version_path, 'train')
eval_log_path = os.path.join(version_path, 'eval')
log_path = version_path

pre_trained_conf_path = os.path.join(pre_trained_models_path, model_tech_name, 'pipeline.config')
model_conf_path = os.path.join(models_path, model_name, version_name, 'pipeline.config')
checkpoint_path = os.path.join(pre_trained_models_path, model_tech_name, 'checkpoint/ckpt-0')

model_name_version = model_name + '_' + version_name
exported_model_path = os.path.join(exported_models_path, model_name_version)
exported_model_conf_path = os.path.join(exported_models_path, model_name_version, 'pipeline.config')

batch_size = 2
num_epoch = 7.5
num_steps = int(num_train_samples / batch_size) * num_epoch
num_steps += 100
print(num_steps)
# ------------------------------------- END MODEL SETTINGS --------------------------------------- #

In [ ]:
!export CUDA_VISIBLE_DEVICES=0
!nvidia-smi -L

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir $log_path/eval

In [ ]:
# VALIDATION - Test set
test_conf_path = version_path + "/test.config"
!python /content/drive/MyDrive/CVproject_Stenosis/workspace/model_main_tf2.py \
  --pipeline_config_path=$test_conf_path \
  --model_dir=$version_path \
  --checkpoint_dir=$version_path \
  --sample_1_of_n_eval_examples=1 \

# EfficientDet D0 512x512 - Testing

## V1

In [ ]:
model_tech_name = 'efficientdet_d0_coco17_tpu-32'
model_name = 'efficientdet_d0'
version_name = 'v1'

model_path = os.path.join(models_path, model_name)
version_path = os.path.join(model_path, version_name)

train_log_path = os.path.join(version_path, 'train')
eval_log_path = os.path.join(version_path, 'eval')
log_path = version_path

pre_trained_conf_path = os.path.join(pre_trained_models_path, model_tech_name, 'pipeline.config')
model_conf_path = os.path.join(models_path, model_name, version_name, 'pipeline.config')
checkpoint_path = os.path.join(pre_trained_models_path, model_tech_name, 'checkpoint/ckpt-0')

model_name_version = model_name + '_' + version_name
exported_model_path = os.path.join(exported_models_path, model_name_version)
exported_model_conf_path = os.path.join(exported_models_path, model_name_version, 'pipeline.config')

batch_size = 4
num_epoch = 15
num_steps = int(num_train_samples / batch_size) * num_epoch
num_steps += 100
print(num_steps)
# ------------------------------------- END MODEL SETTINGS --------------------------------------- #

In [ ]:
!export CUDA_VISIBLE_DEVICES=0
!nvidia-smi -L

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir $log_path/eval

In [ ]:
# VALIDATION
!python /content/drive/MyDrive/CVproject_Stenosis/workspace/model_main_tf2.py \
  --pipeline_config_path=$model_conf_path \
  --model_dir=$version_path \
  --checkpoint_dir=$version_path \
  --sample_1_of_n_eval_examples=1 \

In [ ]:
# VALIDATION - Test set
test_conf_path = version_path + "/test.config"
!python /content/drive/MyDrive/CVproject_Stenosis/workspace/model_main_tf2.py \
  --pipeline_config_path=$test_conf_path \
  --model_dir=$version_path \
  --checkpoint_dir=$version_path/custom_ckpt \
  --sample_1_of_n_eval_examples=1 \

## V2

In [ ]:
model_tech_name = 'efficientdet_d0_coco17_tpu-32'
model_name = 'efficientdet_d0'
version_name = 'v2'

model_path = os.path.join(models_path, model_name)
version_path = os.path.join(model_path, version_name)

train_log_path = os.path.join(version_path, 'train')
eval_log_path = os.path.join(version_path, 'eval')
log_path = version_path

pre_trained_conf_path = os.path.join(pre_trained_models_path, model_tech_name, 'pipeline.config')
model_conf_path = os.path.join(models_path, model_name, version_name, 'pipeline.config')
checkpoint_path = os.path.join(pre_trained_models_path, model_tech_name, 'checkpoint/ckpt-0')

model_name_version = model_name + '_' + version_name
exported_model_path = os.path.join(exported_models_path, model_name_version)
exported_model_conf_path = os.path.join(exported_models_path, model_name_version, 'pipeline.config')

batch_size = 4
num_epoch = 15
num_steps = int(num_train_samples / batch_size) * num_epoch
num_steps += 100
print(num_steps)
# ------------------------------------- END MODEL SETTINGS --------------------------------------- #

In [ ]:
!export CUDA_VISIBLE_DEVICES=0
!nvidia-smi -L

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir $log_path/eval

In [ ]:
# VALIDATION
!python /content/drive/MyDrive/CVproject_Stenosis/workspace/model_main_tf2.py \
  --pipeline_config_path=$model_conf_path \
  --model_dir=$version_path \
  --checkpoint_dir=$version_path \
  --sample_1_of_n_eval_examples=1 \

In [ ]:
# VALIDATION - Test set
test_conf_path = version_path + "/test.config"
!python /content/drive/MyDrive/CVproject_Stenosis/workspace/model_main_tf2.py \
  --pipeline_config_path=$test_conf_path \
  --model_dir=$version_path \
  --checkpoint_dir=$version_path/custom_ckpt \
  --sample_1_of_n_eval_examples=1 \

## V3

In [ ]:
model_tech_name = 'efficientdet_d0_coco17_tpu-32'
model_name = 'efficientdet_d0'
version_name = 'v3'

model_path = os.path.join(models_path, model_name)
version_path = os.path.join(model_path, version_name)

train_log_path = os.path.join(version_path, 'train')
eval_log_path = os.path.join(version_path, 'eval')
log_path = version_path

pre_trained_conf_path = os.path.join(pre_trained_models_path, model_tech_name, 'pipeline.config')
model_conf_path = os.path.join(models_path, model_name, version_name, 'pipeline.config')
checkpoint_path = os.path.join(pre_trained_models_path, model_tech_name, 'checkpoint/ckpt-0')

model_name_version = model_name + '_' + version_name
exported_model_path = os.path.join(exported_models_path, model_name_version)
exported_model_conf_path = os.path.join(exported_models_path, model_name_version, 'pipeline.config')

batch_size = 4
num_epoch = 15
num_steps = int(num_train_samples / batch_size) * num_epoch
num_steps += 100
print(num_steps)
# ------------------------------------- END MODEL SETTINGS --------------------------------------- #

In [ ]:
!export CUDA_VISIBLE_DEVICES=0
!nvidia-smi -L

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir $log_path/eval

In [ ]:
# VALIDATION - Test set
test_conf_path = version_path + "/test.config"
!python /content/drive/MyDrive/CVproject_Stenosis/workspace/model_main_tf2.py \
  --pipeline_config_path=$test_conf_path \
  --model_dir=$version_path \
  --checkpoint_dir=$version_path \
  --sample_1_of_n_eval_examples=1 \

## V4

In [ ]:
model_tech_name = 'efficientdet_d0_coco17_tpu-32'
model_name = 'efficientdet_d0'
version_name = 'v4'

model_path = os.path.join(models_path, model_name)
version_path = os.path.join(model_path, version_name)

train_log_path = os.path.join(version_path, 'train')
eval_log_path = os.path.join(version_path, 'eval')
log_path = version_path

pre_trained_conf_path = os.path.join(pre_trained_models_path, model_tech_name, 'pipeline.config')
model_conf_path = os.path.join(models_path, model_name, version_name, 'pipeline.config')
checkpoint_path = os.path.join(pre_trained_models_path, model_tech_name, 'checkpoint/ckpt-0')

model_name_version = model_name + '_' + version_name
exported_model_path = os.path.join(exported_models_path, model_name_version)
exported_model_conf_path = os.path.join(exported_models_path, model_name_version, 'pipeline.config')

batch_size = 4
num_epoch = 15
num_steps = int(num_train_samples / batch_size) * num_epoch
num_steps += 100
print(num_steps)
# ------------------------------------- END MODEL SETTINGS --------------------------------------- #

In [ ]:
!export CUDA_VISIBLE_DEVICES=0
!nvidia-smi -L

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir $log_path/eval

In [ ]:
# VALIDATION - Test set
test_conf_path = version_path + "/test.config"
!python /content/drive/MyDrive/CVproject_Stenosis/workspace/model_main_tf2.py \
  --pipeline_config_path=$test_conf_path \
  --model_dir=$version_path \
  --checkpoint_dir=$version_path \
  --sample_1_of_n_eval_examples=1 \

# Faster R-CNN ResNet50 V1 640x640 - Testing

## V1

In [ ]:
model_tech_name = 'faster_rcnn_resnet50_v1_640x640_coco17_tpu-8'
model_name = 'faster_rcnn_resnet50'
version_name = 'v1'

model_path = os.path.join(models_path, model_name)
version_path = os.path.join(model_path, version_name)

train_log_path = os.path.join(version_path, 'train')
eval_log_path = os.path.join(version_path, 'eval')
log_path = version_path

pre_trained_conf_path = os.path.join(pre_trained_models_path, model_tech_name, 'pipeline.config')
model_conf_path = os.path.join(models_path, model_name, version_name, 'pipeline.config')
checkpoint_path = os.path.join(pre_trained_models_path, model_tech_name, 'checkpoint/ckpt-0')

model_name_version = model_name + '_' + version_name
exported_model_path = os.path.join(exported_models_path, model_name_version)
exported_model_conf_path = os.path.join(exported_models_path, model_name_version, 'pipeline.config')

batch_size = 2
num_epoch = 7.5
num_steps = int(num_train_samples / batch_size) * num_epoch 
num_steps += 100
print(num_steps)
# ------------------------------------- END MODEL SETTINGS --------------------------------------- #

In [ ]:
!export CUDA_VISIBLE_DEVICES=0
!nvidia-smi -L

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir $log_path/eval

In [ ]:
# VALIDATION - Test Set
test_conf_path = version_path + "/test.config"
!python /content/drive/MyDrive/CVproject_Stenosis/workspace/model_main_tf2.py \
  --pipeline_config_path=$test_conf_path \
  --model_dir=$version_path \
  --checkpoint_dir=$version_path \
  --sample_1_of_n_eval_examples=1

## V2

In [ ]:
model_tech_name = 'faster_rcnn_resnet50_v1_640x640_coco17_tpu-8'
model_name = 'faster_rcnn_resnet50'
version_name = 'v2'

model_path = os.path.join(models_path, model_name)
version_path = os.path.join(model_path, version_name)

train_log_path = os.path.join(version_path, 'train')
eval_log_path = os.path.join(version_path, 'eval')
log_path = version_path

pre_trained_conf_path = os.path.join(pre_trained_models_path, model_tech_name, 'pipeline.config')
model_conf_path = os.path.join(models_path, model_name, version_name, 'pipeline.config')
checkpoint_path = os.path.join(pre_trained_models_path, model_tech_name, 'checkpoint/ckpt-0')

model_name_version = model_name + '_' + version_name
exported_model_path = os.path.join(exported_models_path, model_name_version)
exported_model_conf_path = os.path.join(exported_models_path, model_name_version, 'pipeline.config')

batch_size = 2
num_epoch = 15
num_steps = int(num_train_samples / batch_size) * num_epoch 
num_steps += 100
print(num_steps)
# ------------------------------------- END MODEL SETTINGS --------------------------------------- #

In [ ]:
!export CUDA_VISIBLE_DEVICES=0
!nvidia-smi -L

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir $log_path/eval

In [ ]:
# VALIDATION
!python /content/drive/MyDrive/CVproject_Stenosis/workspace/model_main_tf2.py \
  --pipeline_config_path=$model_conf_path \
  --model_dir=$version_path \
  --checkpoint_dir=$version_path/custom_ckpt \
  --sample_1_of_n_eval_examples=1 \

In [ ]:
# VALIDATION - Test Set
test_conf_path = version_path + "/test.config"
!python /content/drive/MyDrive/CVproject_Stenosis/workspace/model_main_tf2.py \
  --pipeline_config_path=$test_conf_path \
  --model_dir=$version_path \
  --checkpoint_dir=$version_path/custom_ckpt \
  --sample_1_of_n_eval_examples=1 \

## V3

In [ ]:
model_tech_name = 'faster_rcnn_resnet50_v1_640x640_coco17_tpu-8'
model_name = 'faster_rcnn_resnet50'
version_name = 'v3'

model_path = os.path.join(models_path, model_name)
version_path = os.path.join(model_path, version_name)

train_log_path = os.path.join(version_path, 'train')
eval_log_path = os.path.join(version_path, 'eval')
log_path = version_path

pre_trained_conf_path = os.path.join(pre_trained_models_path, model_tech_name, 'pipeline.config')
model_conf_path = os.path.join(models_path, model_name, version_name, 'pipeline.config')
checkpoint_path = os.path.join(pre_trained_models_path, model_tech_name, 'checkpoint/ckpt-0')

model_name_version = model_name + '_' + version_name
exported_model_path = os.path.join(exported_models_path, model_name_version)
exported_model_conf_path = os.path.join(exported_models_path, model_name_version, 'pipeline.config')

batch_size = 2
num_epoch = 7.5
num_steps = int(num_train_samples / batch_size) * num_epoch 
num_steps += 100
print(num_steps)
# ------------------------------------- END MODEL SETTINGS --------------------------------------- #

In [ ]:
!export CUDA_VISIBLE_DEVICES=0
!nvidia-smi -L

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir $log_path/eval

In [ ]:
# VALIDATION - Test Set
test_conf_path = version_path + "/test.config"
!python /content/drive/MyDrive/CVproject_Stenosis/workspace/model_main_tf2.py \
  --pipeline_config_path=$test_conf_path \
  --model_dir=$version_path \
  --checkpoint_dir=$version_path \
  --sample_1_of_n_eval_examples=1

## V4

In [ ]:
model_tech_name = 'faster_rcnn_resnet50_v1_640x640_coco17_tpu-8'
model_name = 'faster_rcnn_resnet50'
version_name = 'v4'

model_path = os.path.join(models_path, model_name)
version_path = os.path.join(model_path, version_name)

train_log_path = os.path.join(version_path, 'train')
eval_log_path = os.path.join(version_path, 'eval')
log_path = version_path

pre_trained_conf_path = os.path.join(pre_trained_models_path, model_tech_name, 'pipeline.config')
model_conf_path = os.path.join(models_path, model_name, version_name, 'pipeline.config')
checkpoint_path = os.path.join(pre_trained_models_path, model_tech_name, 'checkpoint/ckpt-0')

model_name_version = model_name + '_' + version_name
exported_model_path = os.path.join(exported_models_path, model_name_version)
exported_model_conf_path = os.path.join(exported_models_path, model_name_version, 'pipeline.config')

batch_size = 2
num_epoch = 7.5
num_steps = int(num_train_samples / batch_size) * num_epoch 
num_steps += 100
print(num_steps)
# ------------------------------------- END MODEL SETTINGS --------------------------------------- #

In [ ]:
!export CUDA_VISIBLE_DEVICES=0
!nvidia-smi -L

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir $log_path/eval

In [ ]:
# VALIDATION - Test Set
test_conf_path = version_path + "/test.config"
!python /content/drive/MyDrive/CVproject_Stenosis/workspace/model_main_tf2.py \
  --pipeline_config_path=$test_conf_path \
  --model_dir=$version_path \
  --checkpoint_dir=$version_path \
  --sample_1_of_n_eval_examples=1

# Plot risultati

In [ ]:
!cp -R /content/drive/MyDrive/CVproject_Stenosis/workspace/models/ssd_resnet50/v1/eval_test /content/plots/test/ssd_v1
!cp -R /content/drive/MyDrive/CVproject_Stenosis/workspace/models/ssd_resnet50/v2/eval_test /content/plots/test/ssd_v2
!cp -R /content/drive/MyDrive/CVproject_Stenosis/workspace/models/efficientdet_d0/v3/eval_test /content/plots/test/ed_v1
!cp -R /content/drive/MyDrive/CVproject_Stenosis/workspace/models/efficientdet_d0/v4/eval_test /content/plots/test/ed_v2
!cp -R /content/drive/MyDrive/CVproject_Stenosis/workspace/models/faster_rcnn_resnet50/v3/eval_test /content/plots/test/frcnn_v1
!cp -R /content/drive/MyDrive/CVproject_Stenosis/workspace/models/faster_rcnn_resnet50/v4/eval_test /content/plots/test/frcnn_v2

!cp -R /content/drive/MyDrive/CVproject_Stenosis/workspace/models/ssd_resnet50/v1/eval_eval /content/plots/eval/ssd_v1
!cp -R /content/drive/MyDrive/CVproject_Stenosis/workspace/models/ssd_resnet50/v2/eval_eval /content/plots/eval/ssd_v2
!cp -R /content/drive/MyDrive/CVproject_Stenosis/workspace/models/efficientdet_d0/v3/eval_eval /content/plots/eval/ed_v1
!cp -R /content/drive/MyDrive/CVproject_Stenosis/workspace/models/efficientdet_d0/v4/eval_eval /content/plots/eval/ed_v2
!cp -R /content/drive/MyDrive/CVproject_Stenosis/workspace/models/faster_rcnn_resnet50/v3/eval_eval /content/plots/eval/frcnn_v1
!cp -R /content/drive/MyDrive/CVproject_Stenosis/workspace/models/faster_rcnn_resnet50/v4/eval_eval /content/plots/eval/frcnn_v2

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir /content/plots/test

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir /content/plots/eval